# Probabilities through monte carlo simulations

Here, rather than training a model to predict the $E[\text{dice roll score}]$, I just build a monte carlo simulation of a Farkle turn. 

In [1]:
import sys
sys.path.insert(0, '../../src')

In [2]:
import random
from collections import defaultdict

from farkle.logic import gameobjects as go

In [32]:
l = []
not l

True

In [47]:
class ExpectedRollScore():
    def __init__(self, num_dice: int):
        self.num_dice = num_dice
        self.obs = 0
        self.possible_score_counts = defaultdict(int)
    
    def score_frequ(self):
        dh: go.DiceHand
        sf = {dh: c / self.obs for dh, c in self.possible_score_counts.items()}
        return sf
    
    def count_score(self, dh: go.DiceHand):
        assert dh.num_dice <= self.num_dice
        self.possible_score_counts[dh] += 1
        
    def log_dicehand(self, dh: go.DiceHand):
        assert len(dh.dice_values_free()) == self.num_dice
        self.obs += 1
        ps = dh.possible_scores()
        if not ps:
            self.count_farkle()
        else:
            for score_hand in ps:
                score_hand: go.DiceHand
                self.count_score(score_hand)
        
    def count_farkle(self):
        self.obs += 1
        self.possible_score_counts['farkle'] += 1
        
    def expected_score(self, points_at_risk: int = 0) -> int:
        dh: go.DiceHand
        s = sum([dh.score * prob if dh != 'farkle' 
                 else -points_at_risk * prob 
                 for dh, prob in self.score_frequ().items()])
        return s
        
    def __repr__(self):
        return f'ExpectedRollScore({self.num_dice})'

In [39]:
dh1 = go.DiceHand()
dh1

0: 4
1: 5
2: 6
3: 1
4: 1
5: 3
Score: 0

In [40]:
dh1.possible_scores()

[0: 1
 Score: 100,
 0: 1
 1: 1
 Score: 200,
 0: 1
 1: 1
 2: 5
 Score: 250,
 0: 1
 1: 5
 Score: 150,
 0: 5
 Score: 50]

In [48]:
ers = ExpectedRollScore(6)

In [49]:
ers.log_dicehand(dh1)

In [50]:
ers.score_frequ()

{0: 1
 Score: 100: 1.0,
 0: 1
 1: 1
 Score: 200: 1.0,
 0: 1
 1: 1
 2: 5
 Score: 250: 1.0,
 0: 1
 1: 5
 Score: 150: 1.0,
 0: 5
 Score: 50: 1.0}

In [51]:
ers.expected_score()

750.0

In [24]:
ers.count_score(dh1)
ers.count_score(go.DiceHand(1,1,1, score=1000))
ers.count_farkle()
ers.count_farkle()
ers.score_frequ()

ers.expected_score(200)

In [21]:
class StatTracker():
    def __init__(self):
        self.epoch = 0
        
        self.dice_to_roll_probs = {i+1: ExpectedRollScore(i+1) for i in range(6)}
        
    def expected_score(self, num_dice: int, points_at_risk: int, depth: int = 5) -> float:
        """
        num_dice: number of dice to roll
        points_at_risk: points lost if Farkle
        depth: number of recursive calls allowed
        """
        ers = self.dice_to_roll
    
    @property
    def expected_hot_dice_score(self) -> float:
        return self.expected_score(num_dice=6)

In [22]:
st = StatTracker()
st.dice_to_roll_probs

{1: ExpectedRollScore(1),
 2: ExpectedRollScore(2),
 3: ExpectedRollScore(3),
 4: ExpectedRollScore(4),
 5: ExpectedRollScore(5),
 6: ExpectedRollScore(6)}

In [25]:
dh1 = go.DiceHand()

In [26]:
dh1

0: 5
1: 1
2: 1
3: 2
4: 4
5: 3
Score: 0

In [30]:
for dh in dh1.possible_scores():
    print(type(dh))
    print(dh)

<class 'farkle.logic.gameobjects.DiceHand'>
0: 1
Score: 100
<class 'farkle.logic.gameobjects.DiceHand'>
0: 1
1: 1
Score: 200
<class 'farkle.logic.gameobjects.DiceHand'>
0: 1
1: 1
2: 5
Score: 250
<class 'farkle.logic.gameobjects.DiceHand'>
0: 1
1: 5
Score: 150
<class 'farkle.logic.gameobjects.DiceHand'>
0: 5
Score: 50


In [31]:


for i in range(15):
    print(i % 6)

0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
